In [31]:
import json
from pycocotools.coco import COCO
import numpy as np
import os
import cv2
import tensorflow as tf

In [28]:
# Paths
train_images_path = 'dataset/train'
train_annotations_path = 'dataset/train/_annotations.coco.json'
valid_images_path = 'dataset/valid'
valid_annotations_path = 'dataset/valid/_annotations.coco.json'
test_images_path = 'dataset/test'
test_annotations_path = 'dataset/test/_annotations.coco.json'

In [34]:
# Generator function to yield batches of data
def coco_data_generator(images_path, annotations_path, batch_size=32):
    coco = COCO(annotations_path)
    image_ids = coco.getImgIds()
    np.random.shuffle(image_ids)
    
    while True:
        for start in range(0, len(image_ids), batch_size):
            end = min(start + batch_size, len(image_ids))
            batch_image_ids = image_ids[start:end]
            images = []
            masks = []
            
            for img_id in batch_image_ids:
                img_info = coco.loadImgs(img_id)[0]
                img_path = os.path.join(images_path, img_info['file_name'])
                image = cv2.imread(img_path)
                
                if image is None:
                    print(f"Image not found at path: {img_path}")
                    continue
                
                images.append(image / 255.0)
                
                ann_ids = coco.getAnnIds(imgIds=img_id)
                anns = coco.loadAnns(ann_ids)
                
                if not anns:
                    print(f"No annotations found for image ID: {img_id}")
                    continue
                
                mask = np.zeros((img_info['height'], img_info['width'], 1))
                
                for ann in anns:
                    if len(ann['segmentation']) == 0:
                        print(f"Skipping annotation {ann['id']} for image ID {img_id} due to empty segmentation")
                        continue
                    try:
                        ann_mask = coco.annToMask(ann)
                        mask = np.maximum(mask, np.expand_dims(ann_mask, axis=-1))
                    except Exception as e:
                        print(f"Error processing annotation {ann['id']} for image ID {img_id}: {e}")
                        continue
                
                masks.append(mask)
            
            yield np.array(images), np.array(masks)

In [35]:
# Define the U-Net model (simplified version)
def unet_model(input_size=(256, 256, 3)):
    inputs = tf.keras.Input(input_size)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    up1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = tf.keras.layers.concatenate([conv2, up1], axis=3)
    conv4 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv4 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up2 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv4)
    up2 = tf.keras.layers.concatenate([conv1, up2], axis=3)
    conv5 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv5 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    conv6 = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = tf.keras.Model(inputs=[inputs], outputs=[conv6])
    return model

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
# Training parameters
batch_size = 8
steps_per_epoch = len(COCO(train_annotations_path).getImgIds()) // batch_size
validation_steps = len(COCO(valid_annotations_path).getImgIds()) // batch_size

# Train the model
train_generator = coco_data_generator(train_images_path, train_annotations_path, batch_size)
valid_generator = coco_data_generator(valid_images_path, valid_annotations_path, batch_size)

model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=valid_generator, validation_steps=validation_steps)

# Evaluate on test data
test_generator = coco_data_generator(test_images_path, test_annotations_path, batch_size)
test_steps = len(COCO(test_annotations_path).getImgIds()) // batch_size

model.evaluate(test_generator, steps=test_steps)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Skipping annotation 2343 for image ID 2339 due to empty segmentation
Skipping annotation 2165 for image ID 2161 due to empty segmentation
Skipping annotation 2701 for image ID 2697 due to empty segmentation
Skipping annotation 2766 for image ID 2762 due to empty segmentation
Skipping annotation 580 for image ID 580 due to empty segmentation
Skipping annotation 625 for image ID 624 due to empty segmentation
Skipping annotation 1214 for image ID 1211 due to empty segmentation
Skipping annotation 1092 for image ID 1089 due to empty segmentation


MemoryError: Unable to allocate 25.0 MiB for an array with shape (8, 640, 640, 1) and data type float64